In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
traditional = pd.read_csv("lineup_data/warriors_playoffs_traditional.csv")
advanced = pd.read_csv("lineup_data/warriors_playoffs_advanced.csv")
misc = pd.read_csv("lineup_data/warriors_playoffs_misc.csv")
opponent = pd.read_csv("lineup_data/warriors_playoffs_opponent.csv")

In [3]:
lineups = traditional.merge(advanced, on=['LINEUPS', 'TEAM', 'GP', 'MIN'], how='inner')
lineups = lineups.merge(misc, on=['LINEUPS', 'TEAM', 'GP', 'MIN'], how='inner')
lineups = lineups.merge(opponent, on=['LINEUPS', 'TEAM', 'GP', 'MIN'], how='inner')

lineups.columns = [c.replace("\n", " ").replace("\xa0", " ") for c in lineups.columns]

lineups = pd.concat([lineups[['LINEUPS']], 
                     lineups.select_dtypes(include='integer')], axis=1) # only int columns

lineups['REB DIFF'] = lineups.REB - lineups['OPP REB']

In [8]:
payton = lineups[lineups.LINEUPS.str.contains("Payton")]
no_payton = lineups[~lineups.LINEUPS.str.contains("Payton")]

payton_on_off = pd.concat([48 * pd.DataFrame(payton.sum()).transpose().iloc[:, 1:] / payton.MIN.sum(),
                           48 * pd.DataFrame(no_payton.sum()).transpose().iloc[:, 1:] / no_payton.MIN.sum()])

payton_on_off.index = ["Payton ON", "Payton OFF"]
payton_on_off['OPP FG%'] = payton_on_off['OPP FGM'] / payton_on_off['OPP FGA']

payton_on_off = payton_on_off[['PTS', 'STL', '+/-_x', 'PTS OFF TO', 
                               'OPP PTS', 'OPP FG%', 'OPP FBPS', 'OPP PITP']]
payton_on_off = payton_on_off.apply(pd.to_numeric).round(1)

payton_on_off#.to_csv("payton.csv")

,PTS,STL,+/-_x,PTS OFF TO,OPP PTS,OPP FG%,OPP FBPS,OPP PITP
Payton ON,109.9,12.0,11.5,23.8,98.4,0.4,8.0,35.8
Payton OFF,112.2,6.7,3.5,14.7,108.7,0.4,12.5,41.6


In [6]:
looney = lineups[lineups.LINEUPS.str.contains("Looney")]
no_looney = lineups[~lineups.LINEUPS.str.contains("Looney")]

looney_on_off = pd.concat([48 * pd.DataFrame(looney.sum()).transpose().iloc[:, 1:] / looney.MIN.sum(),
                           48 * pd.DataFrame(no_looney.sum()).transpose().iloc[:, 1:] / no_looney.MIN.sum()])

looney_on_off.index = ["Looney ON", "Looney OFF"]
looney_on_off['OPP FG%'] = looney_on_off['OPP FGM'] / looney_on_off['OPP FGA']

looney_on_off = looney_on_off[['PTS', 'REB', 'OREB', 'DREB', 'REB DIFF', 
                               '2ND PTS', 'OPP 2ND PTS', '+/-_x', 'PTS OFF TO', 
                               'OPP PTS', 'OPP FG%', 'OPP FBPS', 'OPP PITP']]
looney_on_off = looney_on_off.apply(pd.to_numeric).round(1)

looney_on_off#.to_csv("looney.csv")